# Main Code


In [ ]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)

In [ ]:
!nvidia-smi -L

In [ ]:
!python /content/drive/MyDrive/repos/dcase-2020-baseline/main.py

In [ ]:
import os
os.chdir('/content/drive/MyDrive/repos/dcase-2020-baseline/')

In [ ]:
!pip install numba==0.48
!pip install librosa==0.7.2
!pip install numpy==1.19.2

In [ ]:
!pip install mxnet-cu80

In [ ]:
!nvcc --version

# Sentence Similarity


In [ ]:
!pip install -U sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('lighteternal/stsb-xlm-r-greek-transfer')

sentences1 = ['Rain coming down on top of a roof, steadily.',
             'Rain is hitting the top of a roof at a steady pace.',
             'Water flowing in a mechanical water mill and dripping down onto some surface.',
             'Water flows at a constant rate and also drips.',
             'Water from a mechanical mill is flowing and dripping down onto a surface.']

sentences2 = ["βροχη πεφτει δυνατα",
             "βροχη πεφτει δυνατα",
             "βροχη πεφτει δυνατα",
             "βροχη πεφτει δυνατα",
             "βροχη πεφτει δυνατα"]


embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities (clone repo for util functions)
from sentence_transformers import util
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} 		 {} 		 Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))
    
#Outputs:
#Το κινητό έπεσε και έσπασε. 		 H πτώση κατέστρεψε τη συσκευή. 		 Score: 0.6741
#Το κινητό έπεσε και έσπασε. 		 Το αυτοκίνητο έσπασε στα δυο. 		 Score: 0.5067
#Το κινητό έπεσε και έσπασε. 		 Ο υπουργός έπεσε και έσπασε το πόδι του. 		 Score: 0.4548


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
model = SentenceTransformer('lighteternal/stsb-xlm-r-greek-transfer')

full_file = pd.read_csv(r'/content/drive/MyDrive/generated_captions.csv',  encoding='utf-8-sig')

print(full_file.shape)


In [ ]:
import numpy as np
import math
import pandas as pd
# print(full_file.iloc[3,0])
file_extend = full_file
print(file_extend.shape)
# file_extend.insert(scores)
print(file_extend.shape)
# for i in range(full_file.shape[0]):
#   print(full_file.iloc[i,0])
list1 = []
list2 = []
list3 = []
adder = 0
# print(full_file.iloc[1,0])
for i in range (full_file.shape[0]):
  list1 = []
  adder = 0 
  mesosoros = 0

  sentences1 = [full_file.iloc[i,2],
               full_file.iloc[i,3],
               full_file.iloc[i,4],
               full_file.iloc[i,5],
               full_file.iloc[i,6]]

  sentences2 = [full_file.iloc[i,0],
               full_file.iloc[i,0],
               full_file.iloc[i,0],
               full_file.iloc[i,0],
               full_file.iloc[i,0]]


  embeddings1 = model.encode(sentences1, convert_to_tensor=True)
  embeddings2 = model.encode(sentences2, convert_to_tensor=True)

  #Compute cosine-similarities (clone repo for util functions)
  from sentence_transformers import util
  cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

  # for i in range(len(sentences1)):
  #   print("{} 		 {} 		 Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))
  for k in range(len(sentences1)):
    # print(sentences1[k])
    # print(sentences2[k])
    # # list2.append(cosine_scores[k][k].format)
    # print("{} ".format(cosine_scores[k][k]))
    # print('score is ', cosine_scores[k][k])
    list1.append(cosine_scores[k][k])
  
    list2.append(cosine_scores[k][k].cpu().detach().numpy())
  print(list1)
  max = 0 
  for indx in range(len(list1)):
    list1[indx]= list1[indx].item()
    list1[indx] = round(list1[indx], 4)
    #if (max < list1[indx]):  #for finding max
      #max = list1[indx]      #
  #list3.append(max)       #


    adder = adder + list1[indx]  # for finding avg
  mesosoros = adder/5 #
  list3.append(mesosoros) #

print(len(list1))
print(mesosoros)
print(list3)


# print(type(cosine_scores))
# print(cosine_scores.shape)
# print(list1)

# # for l in range (len(list1)):
# #   list1[l] = list1[l].score_function

# print (list1)
# print(list2)
# print(len(list1))
# print(len(list2))



# print(list1)

In [ ]:
# for avg
# file_extend.insert(8, "score", list3)

file_extend['score'] = list3



In [ ]:
print(len(list3))

In [ ]:
# for max 
file_extend2 = full_file
print(file_extend2.shape)
file_extend2['max_score'] = list3
print(file_extend2.shape)

file_extend2.to_csv('final_with_max_scores.csv', encoding= 'utf-8')

In [ ]:
print(file_extend.iloc[0,7])
for i in range(file_extend.shape[0]):
      file_extend.iloc[i,7] = round(file_extend.iloc[i,7], 4)


In [ ]:
good = 0
for j in range(file_extend2.shape[0]):
  if (file_extend2.iloc[j,8] >= 0.5):
    good = good +1
print(good)

In [ ]:
file_extend.to_csv('final_with_scores2.csv', encoding= 'utf-8')


In [ ]:
from matplotlib import pyplot as plt

a = file_extend['max_score']
# Creating histogram
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(a, bins = [0,0.1 ,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
 
# Adding extra features   
plt.xlabel("max Sentence Similarity score")
plt.ylabel("number of generated captions")
plt.legend(legend)
plt.title('Histogram of Sentence SImilarity Distribution')
# Show plot
plt.show()